In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns
import plotly.express as px
import functions as funcs
import pyemma as pm
from pandas.api.types import CategoricalDtype
import matplotlib as mpl
import numpy as np
import functions as funcs

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy as sp
import pickle

In [2]:
def zero_var(x):
    if x.dtype=='object':
        return np.unique(x).shape[0] == 1
    else:
        return np.var(x) < 1e-12

# Load data

In [3]:
data_dir = Path('/Volumes/REA/Data/fast_folders/')


Chosen lags and num_dom_procs are the specific values of markov lag time and number of dominant processes we're going to use in this analysis. 

In [4]:
chosen_lags = pd.read_hdf('chosen_lag_times.h5', key='chosen_lags')
chosen_dom_procs = pd.read_hdf('chosen_num_dominant.h5', key='chosen_num_dominant')

# Load, subset and aggregate timescales

'timescales' contains all the timescale data. Subset the timescales for the specific lag and only keep the dominant timescales. 

In [5]:
ts = pd.read_hdf('timescales.h5', key='timescales')

# Subset chosen lag time and number of implied timescales
lags_dict = dict(zip(chosen_lags['protein'], chosen_lags['lag']))
proc_dict = dict(zip(chosen_dom_procs['protein'], chosen_dom_procs['num_its']))

ts['choose_lag'] = ts['protein'].apply(lambda x: lags_dict[x])
ts['choose_k'] = ts['protein'].apply(lambda x: proc_dict[x])
ts = ts.loc[(ts.lag == ts.choose_lag) & (ts.num_its <= ts.choose_k+1), : ]

ts = ts.drop(columns=ts.filter(like='choose', axis=1).columns)
ts = ts.drop(columns=ts.columns[ts.apply(zero_var, axis=0)])

# aggregate
non_num_cols = list(ts.columns[ts.dtypes == 'object'])
agg_columns = ['protein', 'num_its', 'hp_index']

tmp = ts.groupby(agg_columns, as_index=False).median()
tmp2 = ts.groupby(agg_columns, as_index=False).first()

ts = tmp.merge(tmp2.loc[:, list(set(non_num_cols+agg_columns))], on=agg_columns, how='left')
ts = ts.drop(columns=['iteration'])
ts.rename(columns={'value': 'timescale'}, inplace=True)
ts.head()

,protein,num_its,hp_index,timescale,lag,cluster__k,distances__centre,distances__steepness,tica__dim,tica__lag,feature__value,distances__transform,distances__scheme
0,BBA,2.0,0.0,1501.259543,40.0,191.0,1.402167,23.955992,8.0,76.0,dihedrals,logistic,ca
1,BBA,2.0,1.0,1235.997261,40.0,154.0,0.686566,41.734560,5.0,2.0,dihedrals,linear,closest-heavy
2,BBA,2.0,2.0,1865.390819,40.0,273.0,0.837282,12.764260,2.0,5.0,dihedrals,logistic,ca
3,BBA,2.0,3.0,1697.786408,40.0,249.0,1.178747,4.835889,1.0,94.0,dihedrals,logistic,ca
4,BBA,2.0,4.0,2265.535678,40.0,270.0,0.424372,9.323621,9.0,23.0,distances,linear,ca


## Load and aggregate VAMP scores

In [6]:
vamps = pd.read_hdf('vamps_and_hps.h5', key='vamps_hps')

vamps = vamps.drop(columns=vamps.columns[vamps.apply(zero_var, axis=0)])
non_num_cols = list(vamps.columns[vamps.dtypes == 'object'])
agg_columns = ['protein', 'hp_index']

tmp = vamps.groupby(agg_columns, as_index=False).median()  # aggregate numeric columns
tmp2 = vamps.groupby(agg_columns, as_index=False).first()  # aggregate all columns

vamps = tmp.merge(tmp2.loc[:, list(set(non_num_cols+agg_columns))], on=agg_columns, how='left')
vamps.rename(columns={'value': 'vamp'}, inplace=True)
vamps.head()

,protein,hp_index,vamp,lag,k,iteration,cluster__k,distances__centre,distances__steepness,tica__dim,tica__lag,choose_lag,choose_k,protein_dir,feature,feature__value,distances__transform,distances__scheme
0,BBA,0.0,3.721257,40.0,4.0,50.5,191.0,1.402167,23.955992,8.0,76.0,40.0,4.0,1fme,Dihedral angles,dihedrals,logistic,ca
1,BBA,1.0,3.488178,40.0,4.0,50.5,154.0,0.686566,41.734560,5.0,2.0,40.0,4.0,1fme,Dihedral angles,dihedrals,linear,closest-heavy
2,BBA,2.0,3.368452,40.0,4.0,50.5,273.0,0.837282,12.764260,2.0,5.0,40.0,4.0,1fme,Dihedral angles,dihedrals,logistic,ca
3,BBA,3.0,2.909253,40.0,4.0,50.5,249.0,1.178747,4.835889,1.0,94.0,40.0,4.0,1fme,Dihedral angles,dihedrals,logistic,ca
4,BBA,4.0,3.860080,40.0,4.0,50.5,270.0,0.424372,9.323621,9.0,23.0,40.0,4.0,1fme,Contact distance,distances,linear,ca


Check the number of cases is the same between the two datasets (ts contains many different timescales, so select the first one)

In [7]:
vamps.shape[0] == ts.loc[ts.num_its == 2, :].shape[0]

True

naming dictionary (for saving files)

In [8]:
prot_dict = dict((x[0][0], x[0][1]) for x in zip(vamps.loc[:, ['protein', 'protein_dir']].drop_duplicates().values))

## Calculate sensitivity to ouptuts

### Choose protein and feature

In [9]:
def fit(data, dep_var, ind_vars, formula, input_space):
    # determin min/max values for scaling function
    dep_range = np.array([data[dep_var].min(), data[dep_var].max()])    
    output_space = {'dep_var': dep_range}
    var_space = input_space.copy()
    var_space.update({dep_var: output_space['dep_var']})
    
    # Create scaler
    vs = funcs.create_grid(var_space)
    vs_y, vs_X = funcs.create_dmatrices(vs, formula=formula)
    _, scaler = funcs.scale_dmatrix(pd.concat([vs_y, vs_X], axis=1), scaler=None)

    # Scale data
    y, X = funcs.create_dmatrices(data, formula=formula)
    data_s, _ = funcs.scale_dmatrix(pd.concat([y, X], axis=1), scaler=scaler)
    # GP data and priors
    dep_var_cols = [x for x in data_s.columns if dep_var in x]
    ind_var_cols = [x for x in data_s.columns if np.any([y in x for y in ind_vars])]
    y = data_s.loc[:, dep_var_cols]
    X =  data_s.loc[:, ind_var_cols]

    l_prior = funcs.gamma(2, 0.5)
    eta_prior = funcs.hcauchy(2)
    sigma_prior = funcs.hcauchy(2)
    
    
    gp, trace, model = funcs.fit_gp(y=y, X=X,  # Data
                                    l_prior=l_prior, eta_prior=eta_prior, sigma_prior=sigma_prior,  # Priors
                                    kernel_type='exponential',  # Kernel
                                    prop_Xu=None,  # proportion of data points which are inducing variables.
                                    bayes_kws=dict(draws=5000, tune=3000, chains=4, cores=4, target_accept=0.90))  # Bayes kws

    results = {'gp': gp, 'trace': trace, 'model': model, 'data': data_s}
    return results


def get_data(data_sets, dep_var, ind_vars, protein, feature, num_its=None, transform=None):
    
    data = data_sets[dep_var].copy()
    ix = (data.protein==protein) & (data.feature__value==feature)
    if dep_var == 'timescale':
        ix = ix & (data.num_its == num_its)
    if feature == 'distances':
        if transform is None:
            raise ValueError('For distance feature you must specify a transform')
        ix = ix & (data.distances__transform == transform)
        
    data = data.loc[ix, [dep_var]+ind_vars]
    return data
 

In [10]:
# feature = 'distances'
# feature_label = 'distances_linear'
# transform = 'linear'
# ind_vars = ['cluster__k', 'tica__dim', 'tica__lag', 'distances__scheme']
# input_space = {'tica__lag': np.array([1, 10, 100]), 'tica__dim': np.array([1, 5, 10]), 'cluster__k': np.array([10, 250, 500]), 
#               'distances__scheme': np.array(['ca', 'closest-heavy'])}


# feature = 'dihedrals'
# feature_label = 'dihedrals'
# transform = None
# ind_vars = ['cluster__k', 'tica__dim', 'tica__lag']
# input_space = {'tica__lag': np.array([1, 10, 100]), 'tica__dim': np.array([1, 5, 10]), 'cluster__k': np.array([10, 250, 500])}


# feature = 'distances'
# feature_label = 'distances_logistic'
# transform = 'logistic'
# ind_vars = ['cluster__k', 'tica__dim', 'tica__lag', 'distances__scheme', 'distances__centre', 'distances__steepness']
# input_space = {'tica__lag': np.array([1, 10, 100]), 'tica__dim': np.array([1, 5, 10]), 'cluster__k': np.array([10, 250, 500]), 
#               'distances__scheme': np.array(['ca', 'closest-heavy']), 
#                'distances__centre': np.array([0.3, 1.5]), 
#                'distances__steepness': np.array([0.1, 50])}

In [11]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

out_dir = Path('sensitivities_exp_log_outcome')
out_dir.mkdir(exist_ok=True)

dep_vars = ['vamp', 'timescale']
data_sets = {'vamp': vamps, 'timescale': ts}
proteins = ts.protein.unique()

feature = 'distances'
feature_label = 'distances_linear'
transform = 'linear'
ind_vars = ['cluster__k', 'tica__dim', 'tica__lag', 'distances__scheme']
input_space = {'tica__lag': np.array([1, 10, 100]), 'tica__dim': np.array([1, 5, 10]), 'cluster__k': np.array([10, 250, 500]), 
              'distances__scheme': np.array(['ca', 'closest-heavy'])}

for protein in proteins:
    dep_var = 'vamp'
    filename = f"{prot_dict[protein]}_{feature_label}_{dep_var}_sensitivity.pkl"
    print(filename)
    formula = f"np.log({dep_var}) ~ 0 + " + ' + '.join(ind_vars)
    data = get_data(data_sets, dep_var, ind_vars, protein, feature, transform=transform)
    results = fit(data, dep_var, ind_vars, formula, input_space)
    
    out_file = out_dir.joinpath(filename)
    if out_file.exists():
        raise RuntimeError(f'{out_file} already exists')    
    pickle.dump(file=out_file.open('wb'), obj=results)
    
    dep_var = 'timescale'
    
    formula = f"np.log({dep_var}) ~ 0 + " + ' + '.join(ind_vars)
    max_its = proc_dict[protein]
    for num_its in range(2, max_its+1):
        filename = f"{prot_dict[protein]}_{feature_label}_{dep_var}_its_{num_its}_sensitivity.pkl"
        print(filename)
        data = get_data(data_sets, dep_var, ind_vars, protein, feature, num_its, transform=transform)
        results = fit(data, dep_var, ind_vars, formula, input_space)
        out_file = out_dir.joinpath(filename)
        if out_file.exists():
            raise RuntimeError(f'{out_file} already exists')
        pickle.dump(file=out_file.open('wb'), obj=results)


1fme_distances_linear_vamp_sensitivity.pkl


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_n, l_tica__lag_s, l_tica__dim_s, l_cluster__k_s, l_distances__scheme[closest-heavy]_s, l_distances__scheme[ca]_s, eta]


INFO (theano.gof.compilelock): Waiting for existing lock by process '16654' (I am process '16680')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/robertarbon/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.8.0-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '16654' (I am process '16635')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/robertarbon/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.8.0-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '16680' (I am process '16635')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/robertarbon/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.8.0-64/lock_dir
/Users/robertarbon/opt/miniconda3/envs/msmsense/lib/python3.8/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `

1fme_distances_linear_timescale_its_2_sensitivity.pkl


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_n, l_tica__lag_s, l_tica__dim_s, l_cluster__k_s, l_distances__scheme[closest-heavy]_s, l_distances__scheme[ca]_s, eta]


...

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_n, l_tica__lag_s, l_tica__dim_s, l_cluster__k_s, l_distances__scheme[closest-heavy]_s, l_distances__scheme[ca]_s, eta]


INFO (theano.gof.compilelock): Waiting for existing lock by process '24497' (I am process '24453')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/robertarbon/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.8.0-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '24472' (I am process '24529')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/robertarbon/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.8.0-64/lock_dir
/Users/robertarbon/opt/miniconda3/envs/msmsense/lib/python3.8/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/Users/robertarbon/opt/miniconda3/envs/msmsense/lib/python3.8/site-packages/th

gtt_distances_linear_timescale_its_4_sensitivity.pkl


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_n, l_tica__lag_s, l_tica__dim_s, l_cluster__k_s, l_distances__scheme[closest-heavy]_s, l_distances__scheme[ca]_s, eta]


INFO (theano.gof.compilelock): Waiting for existing lock by process '24717' (I am process '24657')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/robertarbon/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.8.0-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '24682' (I am process '24657')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/robertarbon/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.8.0-64/lock_dir
/Users/robertarbon/opt/miniconda3/envs/msmsense/lib/python3.8/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/Users/robertarbon/opt/miniconda3/envs/msmsense/lib/python3.8/site-packages/th